In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import random


In [2]:
# Data pre-processing
file_path='total_new.csv'
df=pd.read_csv(file_path)
df['Metadata_experiment_type'].value_counts()
df["Metadata_gene"]=df["Metadata_gene"].fillna('empty')
df_fill_empty=df
df_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']=='negcon']
df_rm_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']!='negcon']
df_rm_empty_gene=df_fill_empty[df_fill_empty['Metadata_gene']!='empty']
df_rm_orf=df_rm_empty_gene[df_rm_empty_gene["Metadata_experiment_type"]!='ORF']

# sampling
df_proceed=df_rm_orf[~df_rm_orf['Metadata_pert_type'].isin(['control'])]
#A549
df_train=df_proceed[df_proceed['Metadata_Plate'].isin(
['BR00116991',
 'BR00116992',
 'BR00116993',
 'BR00116994',
 'BR00117015',
 'BR00117016',
 'BR00117017',
 'BR00117019',
 'BR00117000',
 'BR00117003',
 'BR00117004',
 'BR00117005',
 'BR00118041',
 'BR00118042',
 'BR00118043',
 'BR00118044'])]
df_test=df_proceed[df_proceed['Metadata_Plate'].isin(['BR00116995',
 'BR00117010',
 'BR00117011',
 'BR00117012',
 'BR00117013',
 'BR00117024',
 'BR00117025',
 'BR00117026',
 'BR00116996',
 'BR00116997',
 'BR00116998',
 'BR00116999',
 'BR00118045',
 'BR00118046',
 'BR00118047',
 'BR00118048'])]
# Train_test pre-process
y_train=df_train['Metadata_gene']
trainX=df_train.drop('Metadata_gene', axis = 1)
testX=df_test.drop('Metadata_gene', axis = 1)
y_test = df_test['Metadata_gene']
trainX=trainX.iloc[:,trainX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
trainX=np.array(trainX)
testX=np.array(testX)
trainY=np.array(y_train)
testY=np.array(y_test)

/tmp/ipykernel_726729/197006386.py:3: DtypeWarning: Columns (2,6,7,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file_path)


In [3]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, recall_score

# Scale the training and test sets using StandardScaler
scaler = StandardScaler()
trainX_scaled = scaler.fit_transform(trainX)
testX_scaled = scaler.transform(testX)

In [4]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, recall_score

# Scale the training and test sets using StandardScaler
scaler = StandardScaler()
trainX_scaled = scaler.fit_transform(trainX)
testX_scaled = scaler.transform(testX)

# Train the MLPClassifier model
mlp_clf = MLPClassifier(hidden_layer_sizes=(1280, 630, 300),
                        max_iter=30, activation='relu', verbose=True, tol=1e-8,
                        solver='adam')
mlp_clf.fit(trainX_scaled, trainY)

Iteration 1, loss = 4.55004776
Iteration 2, loss = 3.57299478
Iteration 3, loss = 2.85163486
Iteration 4, loss = 2.30075802
Iteration 5, loss = 1.84424412
Iteration 6, loss = 1.43686592
Iteration 7, loss = 1.14201164
Iteration 8, loss = 0.84288184
Iteration 9, loss = 0.62427188
Iteration 10, loss = 0.45967218
Iteration 11, loss = 0.35195854
Iteration 12, loss = 0.22348271
Iteration 13, loss = 0.17094062
Iteration 14, loss = 0.16707674
Iteration 15, loss = 0.12653758
Iteration 16, loss = 0.10160247
Iteration 17, loss = 0.08870851
Iteration 18, loss = 0.07381332
Iteration 19, loss = 0.06801078
Iteration 20, loss = 0.03601895
Iteration 21, loss = 0.03117551
Iteration 22, loss = 0.01653487
Iteration 23, loss = 0.01750765
Iteration 24, loss = 0.01671101
Iteration 25, loss = 0.01209541
Iteration 26, loss = 0.02427450
Iteration 27, loss = 0.01358173
Iteration 28, loss = 0.01050004
Iteration 29, loss = 0.00847682
Iteration 30, loss = 0.00649320


/home/yuchen.yang@insilico.ai/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(1280, 630, 300), max_iter=30, tol=1e-08,
              verbose=True)

These below are MLP Classification

In [5]:
# plug in mode = 'top_1' if you want to apply normal MLP classification
# plug in mode = 'top_10' if you want to get the top 10 highest 
def Compute_MLP_Result(trainX,testX,testY,trainY,mode):
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import accuracy_score
    import numpy as np
    from sklearn.neural_network import MLPClassifier
    from sklearn.metrics import classification_report, recall_score
    if mode == 'top_1':
    # MLP TOP ONE (Not TOP TEN)
        sc=StandardScaler()
        scaler = sc.fit(trainX)
        trainX_scaled = scaler.transform(trainX)
        testX_scaled = scaler.transform(testX)
        y_pred = mlp_clf.predict(testX_scaled)
    #print('Seen')
    #print(classification_report(testY, y_pred))
        report=classification_report(testY, y_pred,output_dict=True)
        result=pd.DataFrame(report).transpose()[['recall']]
        result.rename(columns={'recall': 'Recall'},inplace=True)
        result.drop(['micro avg','macro avg','weighted avg'],axis=0,errors='ignore',inplace = True)
        result.index.name='Label'
        result=result[result.index.isin(pd.Series(testY).unique())]
        return result
    if mode =='top_10':
        # MLP Top Ten

        from sklearn.preprocessing import LabelEncoder
        sc = StandardScaler()
        sc.fit(testX)
        le = LabelEncoder()
        trainY_encoded = le.fit_transform(trainY)
        testY_encoded = le.transform(testY)
        y_pred_probs = mlp_clf.predict_proba(sc.transform(testX))

# Initialize a dictionary to keep track of the number of correct predictions for each label
        label_counts = {label: {'total': 0, 'correct': 0} for label in set(testY)}

# Loop over each sample in the test set
        for i in range(len(testY)):
            true_label = testY[i]
            true_label_encoded = testY_encoded[i]
            top_ten_labels_encoded = np.argsort(y_pred_probs[i])[::-1][:10]
            top_ten_labels = le.inverse_transform(top_ten_labels_encoded)
            if true_label in top_ten_labels:
                label_counts[true_label]['correct'] += 1
            label_counts[true_label]['total'] += 1

# Calculate the accuracy for each label and print the results
        for label, counts in label_counts.items():
            if counts['total'] > 0:
                accuracy = counts['correct'] / counts['total']
                print(f"Label {label}: {accuracy:.2f} ({counts['correct']}/{counts['total']})")
            else:
                print(f"Label {label}: No samples in test set")

# Calculate the accuracy for each label and store the results in a dictionary
        accuracy_dict = {}
        for label, counts in label_counts.items():
            if counts['total'] > 0:
                accuracy = counts['correct'] / counts['total']
                accuracy_dict[label] = accuracy

# Create a DataFrame with the accuracy for each label
        accuracy_df = pd.DataFrame.from_dict(accuracy_dict, orient='index', columns=['Accuracy'])
        accuracy_df.index.name = 'Label'
        accuracy_df = accuracy_df.sort_values(by='Accuracy', ascending=False)
        return accuracy_df
        

In [7]:
# top1 / Top 10 Result 
top1_seen_result=Compute_MLP_Result(trainX,testX,testY,trainY,'top_1')
top10_seen_result=Compute_MLP_Result(trainX,testX,testY,trainY,'top_10')

Label SCNN1G: 0.62 (20/32)
Label GAA: 0.23 (7/31)
Label CDC25A: 0.58 (18/31)
Label IL1B: 0.25 (8/32)
Label SLC7A11: 0.19 (6/31)
Label CTSG: 0.28 (9/32)
Label FPR1: 0.31 (10/32)
Label NTRK1: 0.32 (10/31)
Label VEGFA: 0.38 (12/32)
Label TBXAS1: 0.34 (11/32)
Label GRIN2A: 0.16 (5/31)
Label ITGB2: 0.31 (10/32)
Label PPAT: 0.22 (7/32)
Label CYP2A6: 0.28 (9/32)
Label ADRA2B: 0.44 (14/32)
Label UGT1A9: 0.10 (3/30)
Label LPAR1: 0.25 (8/32)
Label PLD1: 0.12 (4/32)
Label HBB: 0.03 (1/32)
Label COMT: 0.66 (21/32)
Label CYP1A2: 0.16 (5/32)
Label KCNK1: 0.12 (4/32)
Label GUCY1B1: 0.29 (9/31)
Label CA5A: 0.17 (5/30)
Label S1PR1: 0.19 (6/31)
Label S1PR2: 0.25 (8/32)
Label PRKCE: 0.37 (11/30)
Label S1PR4: 0.23 (7/30)
Label PDE3A: 0.25 (8/32)
Label HPGDS: 0.26 (8/31)
Label ATP5F1D: 0.42 (13/31)
Label RPL3: 0.56 (18/32)
Label S100B: 0.23 (7/31)
Label KCNN4: 0.39 (12/31)
Label AGER: 0.39 (12/31)
Label CACNA2D3: 0.43 (13/30)
Label AKR1C1: 0.27 (8/30)
Label LCK: 0.81 (26/32)
Label CDK9: 0.38 (12/32)
Label 

In [8]:
print('MLP top1',top1_seen_result['Recall'].mean())
print('MLP top10',top10_seen_result['Accuracy'].mean())

MLP top1 0.05386734904880065
MLP top10 0.3511145574855252


In [9]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

def kNCM(trainX, trainY, testX, testY, k, mode):
    # Convert the label strings to integers
    label_to_int = {label: i for i, label in enumerate(set(trainY))}
    trainY_int = np.array([label_to_int[label] for label in trainY])
    testY_int = np.array([label_to_int[label] for label in testY])

    # Compute the mean vectors for each class in the training set
    class_means = {}
    for label in set(trainY_int):
        class_means[label] = np.mean(trainX[trainY_int == label], axis=0)

    # Use the class means as the label vectors
    label_vectors = np.array(list(class_means.values()))

    # Classify the test samples using the k-nearest class mean algorithm
    knn_clf = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
    knn_clf.fit(label_vectors, list(class_means.keys()))
    nn_indices = knn_clf.kneighbors(testX, return_distance=False)

    # Compute the predicted labels for each test sample
    top_ten_labels_int = np.array([[knn_clf.classes_[i] for i in nn] for nn in nn_indices])

    # Convert the label indices to label names and filter out invalid labels
    int_to_label = {i: label for label, i in label_to_int.items()}
    top_ten_labels = np.vectorize(lambda x: int_to_label.get(x, None))(top_ten_labels_int)
    if mode == 'top_1':
        top_ten_labels = np.array([labels[labels != None][:1] for labels in top_ten_labels])
    if mode == 'top_10':
        top_ten_labels = np.array([labels[labels != None][:10] for labels in top_ten_labels])

    # Merge testY and top_ten_labels into a DataFrame
    df = pd.DataFrame({'testY': testY, 'top_ten_labels': top_ten_labels.tolist()})
    lst=[]
    for i in range(len(df)):
        if df['testY'][i] in df['top_ten_labels'][i]:
            score=1
        else:
            score=0
        lst.append(score)
    df['score']=lst
    avg_df = df.groupby('testY')['score'].mean().reset_index()
    avg_df=avg_df.sort_values(by='score',ascending= False)
    return avg_df

This is for SLPP

In [10]:
# Data pre-processing
file_path='total_new.csv'
df=pd.read_csv(file_path)
df['Metadata_experiment_type'].value_counts()
df["Metadata_gene"]=df["Metadata_gene"].fillna('empty')
df_fill_empty=df
df_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']=='negcon']
df_rm_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']!='negcon']
df_rm_empty_gene=df_fill_empty[df_fill_empty['Metadata_gene']!='empty']
df_rm_orf=df_rm_empty_gene[df_rm_empty_gene["Metadata_experiment_type"]!='ORF']

# sampling
df_proceed=df_rm_orf[~df_rm_orf['Metadata_pert_type'].isin(['control'])]
#A549
df_train=df_proceed[df_proceed['Metadata_Plate'].isin(
['BR00116991',
 'BR00116992',
 'BR00116993',
 'BR00116994',
 'BR00117015',
 'BR00117016',
 'BR00117017',
 'BR00117019',
 'BR00117000',
 'BR00117003',
 'BR00117004',
 'BR00117005',
 'BR00118041',
 'BR00118042',
 'BR00118043',
 'BR00118044'])]
df_test=df_proceed[df_proceed['Metadata_Plate'].isin(['BR00116995',
 'BR00117010',
 'BR00117011',
 'BR00117012',
 'BR00117013',
 'BR00117024',
 'BR00117025',
 'BR00117026',
 'BR00116996',
 'BR00116997',
 'BR00116998',
 'BR00116999',
 'BR00118045',
 'BR00118046',
 'BR00118047',
 'BR00118048'])]
# Train_test pre-process
y_train=df_train['Metadata_gene']
trainX=df_train.drop('Metadata_gene', axis = 1)
testX=df_test.drop('Metadata_gene', axis = 1)
y_test = df_test['Metadata_gene']
trainX=trainX.iloc[:,trainX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
trainX=np.array(trainX)
testX=np.array(testX)
trainY=np.array(y_train)
testY=np.array(y_test)

/tmp/ipykernel_726729/197006386.py:3: DtypeWarning: Columns (2,6,7,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file_path)


In [11]:
n_components=300
sigma=1.0

In [12]:
## code from: github.com/tanyapole/reproduce-OSLPP/blob/main/OSLPP.py
import scipy 

def get_l2_norm(features:np.ndarray): return np.sqrt(np.square(features).sum(axis=1)).reshape((-1,1))

def get_l2_normalized(features:np.ndarray): return features / get_l2_norm(features)

def get_PCA(features, dim):
    result = PCA(n_components=dim).fit_transform(features)
    assert len(features) == len(result)
    return result

def get_W(labels,):
    W = (labels.reshape(-1,1) == labels).astype(np.int32)
    negative_one_idxs = np.where(labels == -1)[0]
    W[:,negative_one_idxs] = 0
    W[negative_one_idxs,:] = 0
    return W

def get_D(W): return np.eye(len(W), dtype=np.int32) * W.sum(axis=1)

def fix_numerical_assymetry(M): return (M + M.transpose()) * 0.5

def get_projection_matrix(features, labels, proj_dim):
    N, d = features.shape
    X = features.transpose()
    
    W = get_W(labels)
    D = get_D(W)
    L = D - W

    A = fix_numerical_assymetry(np.matmul(np.matmul(X, D), X.transpose()))
    B = fix_numerical_assymetry(np.matmul(np.matmul(X, L), X.transpose()) + np.eye(d))
    assert (A.transpose() == A).all() and (B.transpose() == B).all()

    w, v = scipy.linalg.eigh(A, B)
    assert w[0] < w[-1]
    w, v = w[-proj_dim:], v[:, -proj_dim:]
    assert np.abs(np.matmul(A, v) - w * np.matmul(B, v)).max() < 1e-5

    w = np.flip(w)
    v = np.flip(v, axis=1)

    for i in range(v.shape[1]):
        if v[0,i] < 0:
            v[:,i] *= -1
    return v

def project_features(P, features):
    # P: pca_dim x proj_dim
    # features: N x pca_dim
    # result: N x proj_dim
    return np.matmul(P.transpose(), features.transpose()).transpose()

y_train_array = np.array(y_train)
P = get_projection_matrix(trainX, y_train_array, n_components)

In [13]:
# Transform training data feature  from 904 to 300 dimension
transformed_data = np.dot(df_train.iloc[:,15:], P)
new_feature=pd.DataFrame(transformed_data)
trainX1=df_train.iloc[:,:df_train.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
trainX1=trainX1.reset_index()
dftrain_trans =pd.concat([trainX1,new_feature],axis=1)

In [14]:
# Transform SEEN test data feature  from 904 to 300 dimension
transformed_data = np.dot(testX, P)
new_feature=pd.DataFrame(transformed_data)
testX1=df_test.iloc[:,:df_test.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
testX1=testX1.reset_index()
dfnew1 =pd.concat([testX1,new_feature],axis=1)

In [15]:
# Prepare for training KNCM
y_train=dftrain_trans['Metadata_gene']
trainX=dftrain_trans.drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc(0):]
from sklearn.neighbors import KNeighborsClassifier
y_test=dfnew1['Metadata_gene']
testX=dfnew1.drop('Metadata_gene', axis = 1)
testX=testX.iloc[:,testX.columns.get_loc(0):]
trainY=y_train.tolist()
testY=y_test.tolist()

In [16]:
k=130
KNCM_seen_result= kNCM(trainX, trainY, testX, testY, k,'top_10')
KNCM_seen_result['If_seen']='seen'
KNCM_seen_result1= kNCM(trainX, trainY, testX, testY, k,'top_1')
KNCM_seen_result1['If_seen']='seen'

In [17]:
print('top1',KNCM_seen_result1['score'].mean())
print('top10',KNCM_seen_result['score'].mean())

top1 0.03260287427626137
top10 0.19313533912324232


This is for Cellprofiler

In [18]:
# Data pre-processing
file_path='total_new.csv'
df=pd.read_csv(file_path)
df['Metadata_experiment_type'].value_counts()
df["Metadata_gene"]=df["Metadata_gene"].fillna('empty')
df_fill_empty=df
df_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']=='negcon']
df_rm_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']!='negcon']
df_rm_empty_gene=df_fill_empty[df_fill_empty['Metadata_gene']!='empty']
df_rm_orf=df_rm_empty_gene[df_rm_empty_gene["Metadata_experiment_type"]!='ORF']

# sampling
df_proceed=df_rm_orf[~df_rm_orf['Metadata_pert_type'].isin(['control'])]
#A549
df_train=df_proceed[df_proceed['Metadata_Plate'].isin(
['BR00116991',
 'BR00116992',
 'BR00116993',
 'BR00116994',
 'BR00117015',
 'BR00117016',
 'BR00117017',
 'BR00117019',
 'BR00117000',
 'BR00117003',
 'BR00117004',
 'BR00117005',
 'BR00118041',
 'BR00118042',
 'BR00118043',
 'BR00118044'])]
df_test=df_proceed[df_proceed['Metadata_Plate'].isin(['BR00116995',
 'BR00117010',
 'BR00117011',
 'BR00117012',
 'BR00117013',
 'BR00117024',
 'BR00117025',
 'BR00117026',
 'BR00116996',
 'BR00116997',
 'BR00116998',
 'BR00116999',
 'BR00118045',
 'BR00118046',
 'BR00118047',
 'BR00118048'])]
# Train_test pre-process
y_train=df_train['Metadata_gene']
trainX=df_train.drop('Metadata_gene', axis = 1)
testX=df_test.drop('Metadata_gene', axis = 1)
y_test = df_test['Metadata_gene']
trainX=trainX.iloc[:,trainX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
trainX=np.array(trainX)
testX=np.array(testX)
trainY=np.array(y_train)
testY=np.array(y_test)

/tmp/ipykernel_726729/197006386.py:3: DtypeWarning: Columns (2,6,7,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file_path)


In [19]:
k=130
CP_seen_result= kNCM(trainX, trainY, testX, testY, k,'top_10')
CP_seen_result['If_seen']='seen'
CP_seen_result1= kNCM(trainX, trainY, testX, testY, k,'top_1')
CP_seen_result1['If_seen']='seen'

In [20]:
print('top1',CP_seen_result1['score'].mean())
print('top10',CP_seen_result['score'].mean())

top1 0.047547043010752695
top10 0.23492710918114143


In [ ]:
import math
def calculate_std(inputs):
    n = len(inputs)
    mean = sum(inputs) / n
    deviations = [(x - mean) ** 2 for x in inputs]
    variance = sum(deviations) / (n - 1)
    std = math.sqrt(variance)
    return std

In [ ]:
inputs= [0.1215, 0.1121, 0.1039, 0.1009, 0.0904]
sum(inputs)/5

In [ ]:
calculate_std(inputs)

In [ ]:
2/((1/ -0.09290380040157403 )+ (1 / -0.16569525556436412) )

In [ ]:
2/((1/-0.05152552481428767)+ (1 / -0.24113417272278811 ) )

In [ ]:
2/((1/-0.06922879067464711  )+ (1 / -0.2062534037197309 ) )

In [ ]:
2/((1/ -0.07101392671211575  )+ (1 / -0.18153638829389238 ) )

In [ ]:
 2/((1/ -0.062486664265203704    )+ (1 / -0.19275838514599655   ) )

In [ ]:
import math
def calculate_std(inputs):
    n=len(inputs)
    mean = sum(inputs)/n
    deviations = [(x-mean)**2 for x in inputs]
    variance = sum(deviations)/(n-1)
    std = math.sqrt(variance)
    return std

In [ ]:
inputs=[-0.11905471884218397,-0.08490793166781521,-0.10366313324486955,-0.10209143293745622,-0.09437854739753619]
sum(inputs)/5

In [ ]:
calculate_std(inputs)

In [ ]:
 2/((1/ 0.087  )+ (1 / 0.078 ) )